In [1]:
-앙상블 모델은 결정트리(DT)를 기반으로 만들어진 모델들이다
- 앙상블 학습모델(분류 및 회귀 모두사용 가능한 모델)
    :랜덤포레스트(RandomForest : RT)
    : 엑스트라 트리(Extra Trees : ET)
    : 그레디언트 부스팅 (Gradient Boosting : GB)
    

SyntaxError: invalid syntax (Temp/ipykernel_21488/4112067850.py, line 1)

## 랜덤포레스트 (RandomForest)

In [2]:
# - 앙상블 학습모델의 가장 대표적인 모델 -> 무조건 한다 
# - 안정적인 성능으로 널리 사용되고 있음 
# - 과대적합되는 것을 막아준다 
# - 검증 및 테스트 뎅터에서 안정적인 성능을 얻을 수 있다 
# 분류를 할때는 랜덤 포레스트를 먼저 시작 
# 랜덤포레스트느 회귀 분류 모두 가능한 모델이다 
# 회귀도 랜덤포레스트 부터 시작 
# <처리순서>
# - 결정트리를 랜덤하게 만들어 숲을 만든다
# - 훈련데이터에서 랜덤하게 샘플을 추출 
# - 다 사용된 샘플은 다시 훈련데이터에 넣어서 샘플 훈련데이터 생성
# - 추출한 샘플 훈련데이터는 일부 중복된 값들로 추출될 수 있음 

# 랜덤하게 데이터를 추출해서 랜덤하게 훈련 
# 훈련데이터에서 추출해서 학습 후 사용 데이터를 반환하고 -> 해당 데이터를 다시 재사용 => 일부 데이터가 중복 되서 학습하는 경우가 있ㅇ다 
# 이를 부팅 샘플링이라고 한다 
# <부트스트랩 샘플링 Bootstrap Sample>
# - 데이터 원본에서 중복을 허용하여 데이터를 샘플링하는 방식 
# - 원본에서 램덤하게 샘플을 추출 
# - 사용이 끝난 샘플을 원본에 다시 반환
# - 위의 순서를 반복하여 다시 원본에서 샘플을 추출하는 방식 


In [6]:
# 데이터가져오기 
import pandas as pd 
wine = pd.read_csv('./data/08_wine.csv')
wine.info()

# 독립 종속 분리 
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

print(target.shape, data.shape)

# train, test 분리 
from sklearn.model_selection import train_test_split
train_input, test_input,train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)
print(train_input.shape, train_target.shape)
print (test_input.shape, test_target.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB
(6497,) (6497, 3)
(5197, 3) (5197,)
(1300, 3) (1300,)


In [14]:
# 교차검증을 통해서 랜덤포레스트 모델 사용 
from sklearn.model_selection import cross_validate 
from sklearn.ensemble import RandomForestClassifier 
import numpy as np

#모델 생성하기 
rf = RandomForestClassifier(n_jobs = -1 , random_state =42 )

# 교차검증 
# return_train_score : 훈련 및 검증결과 보여주기 
scores = cross_validate(rf, train_input, train_target,
                       return_train_score=True, n_jobs = -1 )
print(scores)
#최종 훈련결과 및 검증 결과 : 기본 5개씩 나온다 
train_mean = np.mean(scores["train_score"])
test_mean = np.mean(scores["test_score"])
print (train_mean, test_mean)
# 약간의 과대 적합 발생 

{'fit_time': array([0.68958116, 0.70157242, 0.69422245, 0.68477225, 0.67943764]), 'score_time': array([0.08156347, 0.07944369, 0.0778842 , 0.08035016, 0.0836885 ]), 'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]), 'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])}
0.9973541965122431 0.8905151032797809


In [15]:
# 특성중요도 확인하기
# 랜덤포레스트 자체 훈련시키기 
rf.fit(train_input, train_target)
print(rf.feature_importances_)

#특성의 순서 : 알콜도수, 당도, 농도 
# 특성에 대한 데이터를 랜덤하게 보아서 성능이 골고루 분배되게 된다 
# -> 효과적으로 데이터가 샘플링 된다 

[0.23167441 0.50039841 0.26792718]


In [20]:
# 랜덤 포레스트만의 특징을 살려서 훈련시키기 

# 모델 생성하기 
# - oob_score (Out of Bag) : 샘플링 
#     : 교차검증 중에 훈련에 참여하지 모산 샘플이 발생할 수 있는데, 
#     : 이러한 샘플을 훈련에 참여시킴으로서 훈련의 정확도를 올림 
#      -> 훈련이 끝난 후 참여하지 못한 데이터를 이용해 또 훈련
#     최적화가 잘된다 
rf = RandomForestClassifier(oob_score = True, n_jobs = -1 , random_state= 42)

#훈련 시키기 
rf.fit(train_input, train_target)
print(rf.oob_score_)
print(rf.score(train_input, train_target))
print(rf.score(test_input, test_target))
# 해석 
# oob 가 사용 된 score 비교시애는 
# oob 건증결과(oob_score_)와 테스트 검증 결괄르 비교한다 
# 0.89 vs 0.88로 성능은 높지 않지만, 과대적합은 해소 되었다 

0.8934000384837406
0.996921300750433
0.8892307692307693


## 엑스트라 트리 (Extra Tree)

In [ ]:
# - 랜덤 포레스트와 유사 
# 기본적으로 100개의 결정트리를 훈련
# 랜덤포래스트와의 차이점 
#     : 부트스트랩을 지원하지 않음 
#         훈련데이터 전체를 이용 
#         무작위로 트리 분할 
# - 특성이 많지 않은 경우에는 랜덤포레스트와 큰 차이 없다 
# - 랜덤하게 트리를 분할하기 때문에(알아서), 계산속도가 빠르다 